# RL Baselines Zoo: Training in Colab



Github Repo: [https://github.com/araffin/rl-baselines-zoo](https://github.com/araffin/rl-baselines-zoo)

Stable-Baselines Repo: [https://github.com/hill-a/stable-baselines](https://github.com/hill-a/stable-baselines)

Medium article: [https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82](https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82)

# Install Dependencies



In [ ]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!apt-get update
!apt-get install swig cmake libopenmpi-dev zlib1g-dev ffmpeg freeglut3-dev xvfb
!pip install stable-baselines[mpi] --upgrade
!pip install pybullet
!pip install box2d box2d-kengz pyyaml pytablewriter optuna scikit-optimize
!pip install -U PyYAML

TensorFlow 1.x selected.
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://ppa.launchpad.net/graphics-drivers

## Clone RL Baselines Zoo Repo

In [ ]:
!git clone https://github.com/araffin/rl-baselines-zoo

Cloning into 'rl-baselines-zoo'...
remote: Enumerating objects: 1797, done.
remote: Total 1797 (delta 0), reused 0 (delta 0), pack-reused 1797
Receiving objects: 100% (1797/1797), 375.65 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (1065/1065), done.
Checking out files: 100% (333/333), done.


In [ ]:
cd rl-baselines-zoo/

/content/rl-baselines-zoo


### Record generate CSV FIles of observations and render videos

In [ ]:
# Set up display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
!pip install pyglet==1.4.0  # pyglet v1.4.1 throws an error

     |████████████████████████████████| 1.0MB 7.4MB/s 
  Found existing installation: pyglet 1.5.0
    Uninstalling pyglet-1.5.0:
      Successfully uninstalled pyglet-1.5.0


In [ ]:
%%writefile utils/record_video.py
import os
import argparse

import gym
import numpy as np
from stable_baselines.common.vec_env import VecVideoRecorder, VecFrameStack, VecNormalize

from .utils import ALGOS, create_test_env, get_saved_hyperparams, get_latest_run_id, find_saved_model

###added###
import pandas as pd
import json
###
if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument('--env', help='environment ID', type=str, default='CartPole-v1')
    parser.add_argument('-f', '--folder', help='Log folder', type=str, default='trained_agents')
    parser.add_argument('-o', '--output-folder', help='Output folder', type=str, default='logs/videos/')
    parser.add_argument('--algo', help='RL Algorithm', default='ppo2',
                        type=str, required=False, choices=list(ALGOS.keys()))
    parser.add_argument('-n', '--n-timesteps', help='number of timesteps', default=1000,
                        type=int)
    parser.add_argument('--n-envs', help='number of environments', default=1,
                        type=int)
    parser.add_argument('--deterministic', action='store_true', default=False,
                        help='Use deterministic actions')
    parser.add_argument('--seed', help='Random generator seed', type=int, default=0)
    parser.add_argument('--no-render', action='store_true', default=False,
                        help='Do not render the environment (useful for tests)')
    parser.add_argument('--exp-id', help='Experiment ID (default: -1, no exp folder, 0: latest)', default=-1,
                        type=int)
    args = parser.parse_args()

    env_id = args.env
    algo = args.algo
    folder = args.folder
    video_folder = args.output_folder
    seed = args.seed
    deterministic = args.deterministic
    video_length = args.n_timesteps
    n_envs = args.n_envs

    if args.exp_id == 0:
        args.exp_id = get_latest_run_id(os.path.join(folder, algo), env_id)
        print('Loading latest experiment, id={}'.format(args.exp_id))
    # Sanity checks
    if args.exp_id > 0:
        log_path = os.path.join(folder, algo, '{}_{}'.format(env_id, args.exp_id))
    else:
        log_path = os.path.join(folder, algo)

    model_path = find_saved_model(algo, log_path, env_id)

    stats_path = os.path.join(log_path, env_id)
    hyperparams, stats_path = get_saved_hyperparams(stats_path)


    is_atari = 'NoFrameskip' in env_id

    env = create_test_env(env_id, n_envs=n_envs, is_atari=is_atari,
                          stats_path=stats_path, seed=seed, log_dir=None,
                          should_render=not args.no_render, hyperparams=hyperparams)

    model = ALGOS[algo].load(model_path)

    obs = env.reset()

    # Note: apparently it renders by default
    env = VecVideoRecorder(env, video_folder,
                           record_video_trigger=lambda x: x == 0, video_length=video_length,
                           name_prefix="{}-{}".format(algo, env_id))

    env.reset()
    ###   MYCODE   ####
    # create the arrays to save the obs, actions and reward to
    # demensions are depending on the obs- and actionspace
    data_dir = 'data/'
    num_obs = len(env.observation_space.sample())
    obs_all = np.zeros(shape=(args.n_timesteps, num_obs))

    num_actions = len(env.action_space.sample())
    actions_all = np.zeros(shape=(args.n_timesteps, num_actions))

    rewards_all = np.zeros(shape=(args.n_timesteps, 1))
    done_all = np.zeros(shape=(args.n_timesteps, 1))
    cumulative_rewards = np.zeros(shape=(args.n_timesteps, 1))
    episode_reward=0
    ###   END      ####
    
    for _ in range(video_length + 1):
        
        # action = [env.action_space.sample()]
        action, state = model.predict(obs, deterministic=deterministic)
        if isinstance(env.action_space, gym.spaces.Box):
            action = np.clip(action, env.action_space.low, env.action_space.high)
        obs, reward, done, infos_ = env.step(action)
        
        ###   MYCODE   ####
        # my Code to get the actions and observations and save them in above defined lists
        ##new csv stuff
        # record all the datapoints
        if _<10000:
          episode_reward+=reward
          obs_all[_] = obs
          actions_all[_] = action
          rewards_all[_] = reward
          done_all[_] = done
          cumulative_rewards[_] = episode_reward
        
        ###   END      ###

    ###   MYCODE   ####
    ### note that this is one of the ending scenarios and below is another
    ### csv
    np_all = np.column_stack([done_all, rewards_all, cumulative_rewards, obs_all, actions_all])

    columns = ['done', 'rewards', 'cumulative_reward']
    columns.extend(['obs_{}'.format(i) for i in range(num_obs)])
    columns.extend(['action_{}'.format(i) for i in range(num_actions)])
    df_all = pd.DataFrame(np_all, columns=columns)

    dir_path = os.path.join(data_dir, algo)
    data_path = os.path.join(dir_path, '{}_{}.csv'.format(algo, env_id))
    os.makedirs(dir_path, exist_ok=True)
    df_all.to_csv(data_path, index=False)
    ###   END      ###


    # Workaround for https://github.com/openai/gym/issues/893
    if n_envs == 1 and 'Bullet' not in env_id and not is_atari:
        env = env.venv
        # DummyVecEnv
        while isinstance(env, VecNormalize) or isinstance(env, VecFrameStack):
            env = env.venv
        env.envs[0].env.close()
    else:
        # SubprocVecEnv
        env.close()

Overwriting utils/record_video.py


parameters

In [ ]:
#setup
env_id="BipedalWalker-v3"
algos= ["a2c", "trpo", "td3", "sac", "ddpg", "ppo2", "acktr"]
video_steps = 10000

In [ ]:
for algorithm in algos:
  print ("recording video:"+ env_id + " " + algorithm + " " + str(video_steps))
  !python -m utils.record_video --algo $algorithm --env $env_id --exp-id 0 -n $video_steps

recording video:BipedalWalker-v3 a2c 10000
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Loading latest experiment, id=0

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
Loading running average
with params: {'norm_obs': True, 'norm_reward': False}
Loading a model without an environment, this model cannot be trained until it has a valid environment.




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.